In [44]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.stats import t
import math

In [16]:
df = pd.read_csv("/Users/zihanding/Developer/CSE544_Project/3.2_linear_interpolation.csv")
df['Year'] = [df['DATE'][i][0:4] for i in range(len(df))]

In [18]:
df2020 = df[df['Year'] == '2020']
df2021 = df[df['Year'] == '2021']

In [19]:
df2020

,DATE,CPI_all_items,Rent_of_Primary_residence,Monthly_Housing_Cost,CPI_Energy,US_Dollar_Purchasing_power,Year
276,2020-01-01,259.037,346.449,244.361,289.137,38.8,2020
277,2020-02-01,259.248,347.141,244.407,289.781,38.7,2020
278,2020-03-01,258.124,347.474,242.655,290.216,38.7,2020
279,2020-04-01,256.092,347.314,240.364,290.639,39.0,2020
280,2020-05-01,255.868,348.344,242.650,291.709,39.0,2020
281,2020-06-01,256.986,349.214,243.475,293.219,38.8,2020
282,2020-07-01,258.278,350.090,244.046,294.599,38.6,2020
283,2020-08-01,259.411,350.673,244.341,295.437,38.5,2020
284,2020-09-01,260.029,351.302,245.745,297.080,38.4,2020
285,2020-10-01,260.286,352.532,245.584,297.893,38.4,2020


In [20]:
df2021

,DATE,CPI_all_items,Rent_of_Primary_residence,Monthly_Housing_Cost,CPI_Energy,US_Dollar_Purchasing_power,Year
288,2021-01-01,262.650,354.330,246.008,300.382,38.2,2021
289,2021-02-01,263.638,355.015,247.387,300.540,38.0,2021
290,2021-03-01,264.914,354.530,248.984,300.897,37.8,2021
291,2021-04-01,266.670,355.331,251.491,301.819,37.4,2021
292,2021-05-01,268.444,355.663,253.934,303.481,37.1,2021
293,2021-06-01,270.559,357.057,254.974,305.634,36.8,2021
294,2021-07-01,271.764,358.024,255.929,308.023,36.6,2021
295,2021-08-01,272.870,359.380,256.078,309.336,36.6,2021
296,2021-09-01,274.028,362.153,256.881,310.996,36.5,2021
297,2021-10-01,276.522,361.733,258.621,313.592,36.2,2021


### Wald test

Qa: For the rent:  
H0: U20 == U21  
H1: U20 != U21

In [24]:
rent20 = np.array(df2020['Rent_of_Primary_residence'])
rent21 = np.array(df2021['Rent_of_Primary_residence'])

In [27]:
def cal_w(theta0,data):
    x = sum(data)/len(data)
    var = sum([(y-x)**2 for y in data])
    return (x-theta0)/var

In [36]:
diff = rent21 - rent20
wald_rent = cal_w(0,diff)
print('w:', wald_rent)
p_value = 2 * norm.sf(abs(wald_rent))
print('p-value:',p_value)

w: 0.4776320580717168
p-value: 0.31645605473695737


We accept H0 that 2020-2021 the Mean didn't change.

Qb: For the Energy:  
H0: U20 == U21  
H1: U20 != U21


In [37]:
energy20 = np.array(df2020['CPI_Energy'])
energy21 = np.array(df2021['CPI_Energy'])

In [38]:
diff = energy21 - energy20
wald_rent = cal_w(0,diff)
print('w:', wald_rent)
p_value = 2 * norm.sf(abs(wald_rent))
print('p-value:',p_value)

w: 0.19301307277140745
p-value: 0.8469487300055867


We accept H0 that 2020-2021 the Mean didn't change.

### T test

In [41]:
def cal_T(theta0,data1,data2):
    x = sum(data1)/len(data1)-sum(data2)/len(data2)
    var1 = sum([(y-x)**2 for y in data1])
    var2 = sum([(y-x)**2 for y in data2])
    se = math.sqrt(var1/len(data1)+var2/len(data2))
    return x/se


Qa: For the rent:  
H0: U20 == U21  
H1: U20 != U21   

In [53]:
t_val = cal_T(0,rent20,rent21)
print('t:', t_val)
p_value = 2 * t.sf(abs(t_val), len(rent20)+len(rent21)-1) 
print('p-value:',p_value)

t: -0.016794095182120185
p-value: 0.9867457163201817


We cannot reject the H0 that 2020-2021 the CPI rent changed.

In [54]:
t_val = cal_T(0,energy20,energy21)
print('t:', t_val)
p_value = 2 * t.sf(abs(t_val), len(rent20)+len(rent21)-1) 
print('p-value:',p_value)

t: -0.03005960444312835
p-value: 0.9762788372743263


We cannot reject the H0 that 2020-2021 the CPI energy changed.

### Z test

use MME we got the mean of Poisson is lambda. The estimate lambda should be the mean of X.

In [49]:
def cal_Z(theta0,X,Y):
    lambdax = sum(X)/len(X)
    lambday = sum(Y)/len(Y)
    x = sum(X)/len(X) - sum(Y)/len(Y)
    varx = lambdax
    vary = lambday
    se = math.sqrt(varx/len(X) + vary/len(Y))
    return x/se

In [55]:
z_val = cal_Z(0,rent20,rent21)
print('z:', z_val)
p_value = 2*norm.sf(abs(z_val))
print('p-value:',p_value)

z: -1.121553768877482
p-value: 0.26205221771743503


We cannot reject the H0 that 2020-2021 the CPI energy changed.

In [57]:
z_val = cal_Z(0,energy20,energy21)
print('z:', z_val)
p_value = 2*norm.sf(abs(z_val))
print('p-value:',p_value)

z: -1.8862737952940327
p-value: 0.05925806861773789


We cannot reject the H0 that 2020-2021 the CPI energy changed.
